In [1]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install smart_open

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [7]:
import os
import gensim
from gensim import corpora
from pprint import pprint
from gensim.utils import simple_preprocess
from smart_open import smart_open
from gensim import models
import gensim.downloader as api

%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

import nltk
import string
import tqdm

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/aiyappa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/aiyappa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
wv = api.load('word2vec-google-news-300')

In [9]:
reviews_raw = pd.read_csv("data/original_data.csv")
reviews_raw['Text'] = reviews_raw['Text'].fillna('')
reviews_raw['Title'] = reviews_raw['Title'] + ' ' + reviews_raw['Text']

In [10]:
reviews_raw.head()

,Title,Political Lean,Score,Id,Subreddit,URL,Num of Comments,Text,Date Created
0,"No matter who someone is, how they look like, ...",Liberal,1,t5fybt,socialism,https://v.redd.it/ng5fyl7hp2l81,0,,1.646272e+09
1,Biden speech draws 38.2 million U.S. TV viewers,Liberal,6,t5fqdn,democrats,https://www.reuters.com/world/us/biden-speech-...,1,,1.646271e+09
2,State of the union Who watched the state of th...,Liberal,1,t5fj9a,DemocraticSocialism,https://www.reddit.com/r/DemocraticSocialism/c...,1,Who watched the state of the union last night ...,1.646270e+09
3,We Should Just Give Poor People Money,Liberal,7,t5f7n9,SocialDemocracy,https://youtu.be/a80kRjpubG0,3,,1.646270e+09
4,Do it for the Dew,Liberal,6,t5es2c,democrats,https://i.redd.it/drmunn90f2l81.jpg,1,,1.646268e+09


In [11]:
reviews_raw.shape

(12854, 9)

In [12]:
def tokenize_text(raw_text):
    stop_words = set(nltk.corpus.stopwords.words('english'))

    tokenized_str = simple_preprocess(raw_text)
    tokens = [i.lower() for i in tokenized_str if (i not in string.punctuation) and (i not in stop_words)]
    return tokens

In [13]:
reviews_raw['tokenized_title'] = reviews_raw.Title.apply(tokenize_text)
reviews_raw.head()

,Title,Political Lean,Score,Id,Subreddit,URL,Num of Comments,Text,Date Created,tokenized_title
0,"No matter who someone is, how they look like, ...",Liberal,1,t5fybt,socialism,https://v.redd.it/ng5fyl7hp2l81,0,,1.646272e+09,"[matter, someone, look, like, language, speak,..."
1,Biden speech draws 38.2 million U.S. TV viewers,Liberal,6,t5fqdn,democrats,https://www.reuters.com/world/us/biden-speech-...,1,,1.646271e+09,"[biden, speech, draws, million, tv, viewers]"
2,State of the union Who watched the state of th...,Liberal,1,t5fj9a,DemocraticSocialism,https://www.reddit.com/r/DemocraticSocialism/c...,1,Who watched the state of the union last night ...,1.646270e+09,"[state, union, watched, state, union, last, ni..."
3,We Should Just Give Poor People Money,Liberal,7,t5f7n9,SocialDemocracy,https://youtu.be/a80kRjpubG0,3,,1.646270e+09,"[give, poor, people, money]"
4,Do it for the Dew,Liberal,6,t5es2c,democrats,https://i.redd.it/drmunn90f2l81.jpg,1,,1.646268e+09,[dew]


In [14]:
reviews_raw = reviews_raw.applymap(lambda x: x.strip() if isinstance(x, str) else x)
reviews_raw.head()

,Title,Political Lean,Score,Id,Subreddit,URL,Num of Comments,Text,Date Created,tokenized_title
0,"No matter who someone is, how they look like, ...",Liberal,1,t5fybt,socialism,https://v.redd.it/ng5fyl7hp2l81,0,,1.646272e+09,"[matter, someone, look, like, language, speak,..."
1,Biden speech draws 38.2 million U.S. TV viewers,Liberal,6,t5fqdn,democrats,https://www.reuters.com/world/us/biden-speech-...,1,,1.646271e+09,"[biden, speech, draws, million, tv, viewers]"
2,State of the union Who watched the state of th...,Liberal,1,t5fj9a,DemocraticSocialism,https://www.reddit.com/r/DemocraticSocialism/c...,1,Who watched the state of the union last night ...,1.646270e+09,"[state, union, watched, state, union, last, ni..."
3,We Should Just Give Poor People Money,Liberal,7,t5f7n9,SocialDemocracy,https://youtu.be/a80kRjpubG0,3,,1.646270e+09,"[give, poor, people, money]"
4,Do it for the Dew,Liberal,6,t5es2c,democrats,https://i.redd.it/drmunn90f2l81.jpg,1,,1.646268e+09,[dew]


In [15]:
dictionary = corpora.Dictionary(reviews_raw.tokenized_title)
inverse_dictionary = {dictionary.get(key): key for key in dictionary.keys()}

corpus = [dictionary.doc2bow(entry) for entry in reviews_raw.tokenized_title]
tfidf = models.TfidfModel(corpus, smartirs='ntc')
word_to_tfidf = {dictionary.get(id): value for doc in tfidf[corpus] for id, value in doc}

In [16]:
vector_length = 300
f_vec = []

for doc in tfidf[corpus]:
    words = [[dictionary[id], freq] for id, freq in doc]
    sentence_feature = np.zeros(vector_length)
    word_count = 0
    for word in words:
        if word[0] in wv:
            word_count += 1
            word_feature = wv[word[0]] * word[1]
            sentence_feature += word_feature

    sentence_feature = sentence_feature/word_count

    f_vec.append(sentence_feature)

reviews_raw['feature_vector_title'] = f_vec

/tmp/ipykernel_5859/2293496076.py:14: RuntimeWarning: invalid value encountered in true_divide
  sentence_feature = sentence_feature/word_count


In [17]:
reviews_raw.head()

,Title,Political Lean,Score,Id,Subreddit,URL,Num of Comments,Text,Date Created,tokenized_title,feature_vector_title
0,"No matter who someone is, how they look like, ...",Liberal,1,t5fybt,socialism,https://v.redd.it/ng5fyl7hp2l81,0,,1.646272e+09,"[matter, someone, look, like, language, speak,...","[0.0019099171816681821, 0.0064279054705467485,..."
1,Biden speech draws 38.2 million U.S. TV viewers,Liberal,6,t5fqdn,democrats,https://www.reuters.com/world/us/biden-speech-...,1,,1.646271e+09,"[biden, speech, draws, million, tv, viewers]","[0.036623875610530376, 0.03366864534715811, 0...."
2,State of the union Who watched the state of th...,Liberal,1,t5fj9a,DemocraticSocialism,https://www.reddit.com/r/DemocraticSocialism/c...,1,Who watched the state of the union last night ...,1.646270e+09,"[state, union, watched, state, union, last, ni...","[-0.006965968835478027, -0.007696884994705518,..."
3,We Should Just Give Poor People Money,Liberal,7,t5f7n9,SocialDemocracy,https://youtu.be/a80kRjpubG0,3,,1.646270e+09,"[give, poor, people, money]","[0.08058890979737043, 0.02701872866600752, 0.0..."
4,Do it for the Dew,Liberal,6,t5es2c,democrats,https://i.redd.it/drmunn90f2l81.jpg,1,,1.646268e+09,[dew],"[0.0191650390625, -0.034423828125, -0.07373046..."


In [18]:
columns = ['f_title' + str(i) for i in range(1,301)]

data_df_title = reviews_raw.feature_vector_title.apply(pd.Series)

data_df_title.columns = columns

In [19]:
data_df_title.head()

,f_title1,f_title2,f_title3,f_title4,f_title5,f_title6,f_title7,f_title8,f_title9,f_title10,...,f_title291,f_title292,f_title293,f_title294,f_title295,f_title296,f_title297,f_title298,f_title299,f_title300
0,0.001910,0.006428,0.026418,0.039048,-0.038640,0.011113,0.034012,-0.019986,0.010028,0.007120,...,-0.006125,0.014911,-0.017079,0.007582,-0.025027,-0.003020,0.013470,-0.008324,0.007025,0.005741
1,0.036624,0.033669,0.041304,0.044857,0.000242,0.021688,-0.002865,-0.021374,0.055723,0.008878,...,-0.008321,-0.008117,-0.063622,0.012256,0.027464,-0.079240,-0.003485,-0.020103,-0.006999,-0.033010
2,-0.006966,-0.007697,0.015100,0.022621,-0.015299,-0.042518,0.009699,-0.050670,0.053695,0.021251,...,-0.019555,-0.025999,-0.031898,0.005084,-0.004887,-0.025996,-0.016266,-0.044735,-0.006449,-0.022301
3,0.080589,0.027019,0.006564,0.072672,0.014970,0.031774,0.000345,-0.003953,0.024885,0.024878,...,-0.026603,0.062459,-0.062161,-0.037832,-0.014413,0.031207,0.002775,-0.028665,0.024689,-0.054684
4,0.019165,-0.034424,-0.073730,-0.237305,0.341797,0.059082,-0.006622,-0.031128,-0.373047,0.183594,...,0.109375,0.136719,0.421875,-0.404297,-0.023193,0.248047,0.003265,0.048340,-0.186523,0.192383


In [20]:
data_df_title = data_df_title.fillna(0)
data_df_title['upvotes'] = reviews_raw['Score']
data_df_title['num_comments'] = reviews_raw['Num of Comments']
data_df_title.head()

,f_title1,f_title2,f_title3,f_title4,f_title5,f_title6,f_title7,f_title8,f_title9,f_title10,...,f_title293,f_title294,f_title295,f_title296,f_title297,f_title298,f_title299,f_title300,upvotes,num_comments
0,0.001910,0.006428,0.026418,0.039048,-0.038640,0.011113,0.034012,-0.019986,0.010028,0.007120,...,-0.017079,0.007582,-0.025027,-0.003020,0.013470,-0.008324,0.007025,0.005741,1,0
1,0.036624,0.033669,0.041304,0.044857,0.000242,0.021688,-0.002865,-0.021374,0.055723,0.008878,...,-0.063622,0.012256,0.027464,-0.079240,-0.003485,-0.020103,-0.006999,-0.033010,6,1
2,-0.006966,-0.007697,0.015100,0.022621,-0.015299,-0.042518,0.009699,-0.050670,0.053695,0.021251,...,-0.031898,0.005084,-0.004887,-0.025996,-0.016266,-0.044735,-0.006449,-0.022301,1,1
3,0.080589,0.027019,0.006564,0.072672,0.014970,0.031774,0.000345,-0.003953,0.024885,0.024878,...,-0.062161,-0.037832,-0.014413,0.031207,0.002775,-0.028665,0.024689,-0.054684,7,3
4,0.019165,-0.034424,-0.073730,-0.237305,0.341797,0.059082,-0.006622,-0.031128,-0.373047,0.183594,...,0.421875,-0.404297,-0.023193,0.248047,0.003265,0.048340,-0.186523,0.192383,6,1


In [21]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
merged_df_scaled = pd.DataFrame(scaler.fit_transform(data_df_title), columns=data_df_title.columns)
merged_df_scaled.head()

,f_title1,f_title2,f_title3,f_title4,f_title5,f_title6,f_title7,f_title8,f_title9,f_title10,...,f_title293,f_title294,f_title295,f_title296,f_title297,f_title298,f_title299,f_title300,upvotes,num_comments
0,0.509233,0.509796,0.526944,0.317724,0.541685,0.531365,0.546596,0.569515,0.399459,0.430380,...,0.589882,0.458434,0.437497,0.547845,0.622242,0.595791,0.422935,0.378929,0.000040,0.000000
1,0.545879,0.545836,0.542127,0.323522,0.588526,0.540965,0.494582,0.568003,0.447108,0.432901,...,0.546397,0.463378,0.501334,0.446170,0.600539,0.577891,0.405029,0.332789,0.000239,0.000465
2,0.499863,0.491109,0.515401,0.301330,0.569805,0.482679,0.512302,0.536101,0.444994,0.450646,...,0.576037,0.455791,0.461990,0.517195,0.584180,0.540460,0.405730,0.345539,0.000040,0.000465
3,0.592292,0.537038,0.506695,0.351282,0.606270,0.550121,0.499110,0.586974,0.414952,0.455848,...,0.547762,0.410393,0.450405,0.593501,0.608552,0.564881,0.445488,0.306981,0.000279,0.001395
4,0.527448,0.455749,0.424801,0.041910,1.000000,0.574911,0.489282,0.557381,0.000000,0.683473,...,1.000000,0.022727,0.439727,0.882757,0.609180,0.681899,0.175810,0.601163,0.000239,0.000465


In [22]:
merged_df_scaled['lean'] = reviews_raw['Political Lean']
merged_df_scaled['lean'] = (merged_df_scaled['lean'] == 'Conservative').astype(int)
merged_df_scaled.tail()

,f_title1,f_title2,f_title3,f_title4,f_title5,f_title6,f_title7,f_title8,f_title9,f_title10,...,f_title294,f_title295,f_title296,f_title297,f_title298,f_title299,f_title300,upvotes,num_comments,lean
12849,0.499167,0.629316,0.497267,0.317746,0.564916,0.520259,0.465108,0.622934,0.448612,0.415877,...,0.446245,0.427308,0.503302,0.591143,0.477949,0.387050,0.374573,0.00008,0.000465,1
12850,0.512759,0.499246,0.506999,0.330946,0.554817,0.517291,0.523612,0.586927,0.417922,0.431827,...,0.460064,0.447617,0.552013,0.605227,0.581705,0.413979,0.397238,0.00008,0.000465,1
12851,0.498842,0.500097,0.516544,0.297469,0.585950,0.523694,0.536435,0.560696,0.413854,0.425998,...,0.483419,0.459884,0.523343,0.605184,0.598216,0.413023,0.385560,0.00008,0.000000,1
12852,0.558649,0.453515,0.462923,0.282751,0.568499,0.491259,0.557589,0.565740,0.436439,0.386869,...,0.503042,0.476479,0.596381,0.571944,0.552460,0.378905,0.352689,0.00008,0.000000,1
12853,0.512564,0.498995,0.500238,0.297374,0.575141,0.531253,0.520155,0.584469,0.410887,0.417110,...,0.465564,0.455464,0.535811,0.611484,0.573240,0.404706,0.375478,0.00008,0.000000,1


In [23]:
merged_df_scaled.shape

(12854, 303)

In [25]:
merged_df_scaled.to_csv('data/w2vecscaled.csv', index=False)